Video dataset update prediction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os, cv2, time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential

from PIL import Image

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import csv


In [ ]:
import torch
import torchvision

path = f'/python_programming/FYP_2022/viadnet/model/viadnet_addtn.pt'
new_model = torch.load(path)

new_model.eval()

Sequential(
  (0): VisionTransformer(
    (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (encoder): Encoder(
      (dropout): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (encoder_layer_0): EncoderBlock(
          (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attention): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (0): Linear(in_features=768, out_features=3072, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
            (3): Linear(in_features=3072, out_features=768, bias=True)
            (4): Dropout(p=0.0, inplace=False)
          )
        )
        (encoder_layer_1): EncoderBlock(
          (ln_1): LayerNor

In [ ]:
# Predict on a test image with a pretrained viadnet model
import torch
import torchvision
from torchvision import transforms
#import matplotlib.pyplot as plt

from typing import List, Tuple

from PIL import Image


def pred_and_plot_image(
   frame, frame_count, frame_imgs,
    model: torch.nn.Module,
    class_names: List[str],
    image_size: Tuple[int, int] = (224, 224),
    transform: torchvision.transforms = None
):

          # Preprocess the frame
          frame = cv2.resize(frame, (224, 224))  # Resize to match the input size of the model
          frame_img = frame
          #frame = frame / 255.0  # Normalize pixel values to be between 0 and 1

          # Expand dimensions to match the model's expected input shape
          #frame = np.expand_dims(frame, axis=0)

          frame_imgs.append(frame_img)

          # Make predictions
          if transform is not None:
               image_transform = transform
          else:
              image_transform = transforms.Compose(
                  [
                    transforms.ToPILImage(),
                    transforms.Resize(image_size),
                    transforms.ToTensor(),
                    transforms.Normalize(
                         mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                    ),
                  ]
              )

          if not isinstance(frame, Image.Image):
            frame = Image.fromarray(frame)
          ### Predict on image ###

          with torch.inference_mode():
              # Transform and add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
              transformed_image = image_transform(np.array(frame)).unsqueeze(dim=0)

                # Make a prediction on image with an extra dimension and send it to the target device
              target_image_pred = model(transformed_image)#.to(device))

           # Convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
          target_image_pred_probs = torch.softmax(target_image_pred, dim=1)
          target_image_pred_probsval = target_image_pred_probs.max().item()

          # Convert prediction probabilities -> prediction labels
          target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1).item()

          target_image_pred_classname = class_names[target_image_pred_label]

          print('image name:', frame_count, '|', 'Pred label:', target_image_pred_label, '|', 'pred prob:', round(target_image_pred_probsval,3), '|', 'class name:', target_image_pred_classname )

          return target_image_pred_label, round(target_image_pred_probsval,3), target_image_pred_classname




calculate frame properties

In [ ]:
def calculate_frame_difference(frame1, frame2, cur_target_image_pred_label, cur_target_image_pred_probsval, cur_target_image_pred_classname, prev_target_image_pred_label, prev_target_image_pred_probsval, prev_target_image_pred_classname):

    frame1 = np.array(frame1)
    frame2 = np.array(frame2)
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(gray1, gray2)

    # Calculate mean, variance, and skewness of brightness
    mean_brightness1 = round(np.mean(gray1),3)
    variance_brightness1 = round(np.var(gray1),3)
    skewness_brightness1 = round(np.mean((gray1 - mean_brightness1) ** 3) / (np.var(gray1) ** (3/2)),3)

    mean_brightness2 = round(np.mean(gray2),3)
    variance_brightness2 = round(np.var(gray2),3)
    skewness_brightness2 = round(np.mean((gray2 - mean_brightness2) ** 3) / (np.var(gray2) ** (3/2)),3)

    # Calculate differences in mean brightness, variance, and skewness
    mean_brightness_diff = round(np.abs(mean_brightness1 - mean_brightness2),3)
    variance_brightness_diff = round(np.abs(variance_brightness1 - variance_brightness2),3)
    skewness_brightness_diff = round(np.abs(skewness_brightness1 - skewness_brightness2),3)

    # Calculate color histogram difference
    hist_diff = cv2.compareHist(cv2.calcHist([gray1], [0], None, [256], [0, 256]),
                                cv2.calcHist([gray2], [0], None, [256], [0, 256]), cv2.HISTCMP_CORREL)


    print('frame_count:', frame_count, '|', 'previmagefile', prev_frame_count,  '|', 'prev_target_image_pred_label:', prev_target_image_pred_label, '|', 'prev_predval:',prev_target_image_pred_probsval, '|', 'prev_target_image_pred_classname:', prev_target_image_pred_classname, '|', 'mean_brightness1:', mean_brightness1, '|',
          'curimagefile:',  current_frame_count, '|', 'cur_target_image_pred_label:', cur_target_image_pred_label, '|','cur_predval:', cur_target_image_pred_probsval, '|', 'cur_target_image_pred_classname:', cur_target_image_pred_classname, '|', 'mean_brightness2:', mean_brightness2, '|',
          'mean_brightness_diff:', mean_brightness_diff, 'hist_diff:',   hist_diff)

    with open('/python_programming/FYP_2022/viadnet/test_video_output/dove/statpredict_dove_video.csv', 'a', newline ='') as f:
        writer = csv.writer(f)
        data = [frame_count, prev_frame_count, prev_target_image_pred_label, prev_target_image_pred_probsval, prev_target_image_pred_classname,  mean_brightness1, current_frame_count, cur_target_image_pred_label, round(cur_target_image_pred_probsval,3), cur_target_image_pred_classname, mean_brightness2, mean_brightness_diff, variance_brightness1, variance_brightness2,
                variance_brightness_diff, skewness_brightness1, skewness_brightness2, skewness_brightness_diff,  hist_diff]
        writer.writerow(data)

    return frame_diff, hist_diff, mean_brightness1, mean_brightness2, mean_brightness_diff, variance_brightness1, variance_brightness2, variance_brightness_diff, skewness_brightness1, skewness_brightness2, skewness_brightness_diff


def is_mean_scene_change(mean_brightness1, mean_brightness2, min_mean_diff_threshold = 4.00, max_mean_diff_threshold= 60):
    return ((np.abs(mean_brightness1 - mean_brightness2) >= min_mean_diff_threshold and np.abs(mean_brightness1 - mean_brightness2) <= max_mean_diff_threshold))

Add dialogue box

In [ ]:
def add_dialogue_box(frame, text):
    # Define the font and other properties
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    font_thickness = 2
    font_color = (0, 0, 255) #red color #(255, 255, 255)  # White color

    # Get the size of the text
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]

    curframe = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)
    # Calculate the position to place the dialogue box
    text_position = (10, curframe.shape[0] - 10)  # Bottom-left corner with some padding

    # Add the text to the frame
    cv2.putText(curframe, text, text_position, font, font_scale, font_color, font_thickness)
   #cv2.imwrite(f'/python_programming/FYP_2022/viadnet/test3/prediction_frame_output/dovevideo_x/{current_frame_count}.jpg', curframe)#frame_img
    return frame

Update prediction

In [ ]:
def updatepredition(prev_image, current_image, cur_target_image_pred_label, cur_target_image_pred_probsval, cur_target_image_pred_classname, prev_target_image_pred_label, prev_target_image_pred_probsval, prev_target_image_pred_classname):

      update_target_image_pred_label = cur_target_image_pred_label
      update_target_image_pred_probsval = cur_target_image_pred_probsval
      update_target_image_pred_classname = cur_target_image_pred_classname

      with open('/python_programming/FYP_2022/viadnet/test_video_output/dove/updatepredict_dove_video.csv', 'a', newline ='') as f:
              writer = csv.writer(f)
              data = [previmagefile, prev_target_image_pred_label, prev_target_image_pred_probsval, prev_target_image_pred_classname,  mean_brightness1, curimagefile, cur_target_image_pred_label, update_target_image_pred_label, cur_target_image_pred_probsval, cur_target_image_pred_classname, mean_brightness2, mean_brightness_diff, variance_brightness1, variance_brightness2,
                variance_brightness_diff, skewness_brightness1, skewness_brightness2, skewness_brightness_diff,  hist_diff]
              writer.writerow(data)

      return update_target_image_pred_label, update_target_image_pred_probsval, update_target_image_pred_classname


Load video data

In [ ]:
# Specify the path to the video file
video_path = '/python_programming/FYP_2022/video_classification/video_dataset/dove_bill.mp4'
class_names = ['Billboard', 'No-billboard']

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get the frames per second (fps) and frame dimensions
fps = int(cap.get(cv2.CAP_PROP_FPS)) #23.976
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))#1280
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))#720

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Video Information: {width}x{height} pixels, {fps} fps, {total_frames} frames")
frame_count = 1
prev_frame_count = 1
current_frame_count = 2


ret, prev_frame = cap.read()

print('frame_count:', frame_count)
print('prev frame count:', prev_frame_count)
print('current frame count:', current_frame_count)

prev_target_image_pred_label = 0
prev_target_image_pred_probsval = 0
prev_target_image_pred_classname = None

mean_brightness_list1 = []
mean_brightness_list2 = []
mean_brightness_diff_list = []
frame_imgs = []

#header = ['frame_count', 'prev_frame_count', "Prev Pred label", "Prev Pred Prob", "prev class name", 'mean_brightness1', 'current_frame_count',"curimagefile", "Cur Pred label",  "Cur Pred Prob", " cur class name" 'mean_brightness2', 'mean_brightness_diff', 'variance_brightness1',
          #'variance_brightness2', 'variance_brightness_diff', 'skewness_brightness1', 'skewness_brightness2', 'skewness_brightness_diff', 'hist_diff']

header = [ 'prev_frame_count', "Prev Pred label", "Prev Pred Prob", "prev class name", 'mean_brightness1', "curimagefile", "Cur Pred label", 'update_pred_label', "Cur Pred Prob", " cur class name", 'mean_brightness2', 'mean_brightness_diff', 'variance_brightness1',
          'variance_brightness2', 'variance_brightness_diff', 'skewness_brightness1', 'skewness_brightness2', 'skewness_brightness_diff', 'hist_diff']

# open the file in the write mode
with open('/python_programming/FYP_2022/viadnet/test_video_output/dove/updatepredict_dove_video.csv', 'a', newline ='') as f:
    writer = csv.writer(f)
    writer.writerow(header)


Video Information: 1280x720 pixels, 23 fps, 2847 frames
frame_count: 1
prev frame count: 1
current frame count: 2


In [ ]:
while True:


    ret, current_frame =  cap.read()

    cur_target_image_pred_label, cur_target_image_pred_probsval, cur_target_image_pred_classname = pred_and_plot_image(prev_frame, frame_count, frame_imgs, model=new_model, class_names=class_names)
    frame_diff, hist_diff, mean_brightness1, mean_brightness2, mean_brightness_diff, variance_brightness1, variance_brightness2, variance_brightness_diff, skewness_brightness1, skewness_brightness2, skewness_brightness_diff = calculate_frame_difference(prev_frame, current_frame, cur_target_image_pred_label, cur_target_image_pred_probsval, cur_target_image_pred_classname, prev_target_image_pred_label, prev_target_image_pred_probsval, prev_target_image_pred_classname)

    print('frame_count:', frame_count)
    print('prev frame count:', prev_frame_count)
    print('current frame count:', current_frame_count)


    mean_brightness_list1.append(mean_brightness1)
    mean_brightness_list2.append(mean_brightness2)
    mean_brightness_diff_list.append(mean_brightness_diff)


    if is_mean_scene_change(mean_brightness1, mean_brightness2, min_mean_diff_threshold = 4.00, max_mean_diff_threshold= 60):

         # Calculate the frame difference
        cur_target_image_pred_label, cur_target_image_pred_probsval, cur_target_image_pred_classname = pred_and_plot_image(current_frame, frame_count, frame_imgs, model=new_model, class_names=class_names)
        #frame_diff, hist_diff, mean_brightness1, mean_brightness2, mean_brightness_diff, variance_brightness1, variance_brightness2, variance_brightness_diff, skewness_brightness1, skewness_brightness2, skewness_brightness_diff = calculate_frame_difference(prev_frame, current_frame, cur_target_image_pred_label, cur_target_image_pred_probsval, cur_target_image_pred_classname, prev_target_image_pred_label, prev_target_image_pred_probsval, prev_target_image_pred_classname)
        frame_with_dialogue = add_dialogue_box(current_frame, f'Frame: {current_frame_count}, Predicted: {cur_target_image_pred_label}, Class:{cur_target_image_pred_classname}')

        print(f"Saving frame{prev_frame_count}  with mean brightness {mean_brightness1}")
        print(f"Saving frame{current_frame_count}  with mean brightness {mean_brightness2}")

    else:
        print(f"Saving frame{prev_frame_count}  with mean brightness {mean_brightness1}")
        print(f"Saving frame{current_frame_count}  with mean brightness {mean_brightness2}")

          ##for no-billboard update prediction
        if (prev_target_image_pred_label == 1 and cur_target_image_pred_label == 0 and mean_brightness_diff<4):

        #update_target_image_pred_label, update_target_image_pred_probsval, update_target_image_pred_classname = updatepredition(prev_image, current_image, cur_target_image_pred_label, cur_target_image_pred_probsval, cur_target_image_pred_classname, prev_target_image_pred_label, prev_target_image_pred_probsval, prev_target_image_pred_classname)

               update_target_image_pred_label = prev_target_image_pred_label
               update_target_image_pred_probsval = prev_target_image_pred_probsval
               update_target_image_pred_classname = prev_target_image_pred_classname

               frame_with_dialogue = add_dialogue_box(current_frame, f'Frame: {current_frame_count}, Predicted: {update_target_image_pred_label}, Class:{update_target_image_pred_classname}')


               with open('/python_programming/FYP_2022/viadnet/test_video_output/dove/updatepredict_dove_video.csv', 'a', newline ='') as f:
                     writer = csv.writer(f)
                     data = [prev_frame_count, prev_target_image_pred_label, prev_target_image_pred_probsval, prev_target_image_pred_classname,  mean_brightness1, current_frame_count, cur_target_image_pred_label, update_target_image_pred_label, update_target_image_pred_probsval, update_target_image_pred_classname, mean_brightness2, mean_brightness_diff, variance_brightness1, variance_brightness2,
                       variance_brightness_diff, skewness_brightness1, skewness_brightness2, skewness_brightness_diff,  hist_diff]
                     writer.writerow(data)

        elif(prev_target_image_pred_label == 0 and cur_target_image_pred_label == 1 and mean_brightness_diff<4):

             #update_target_image_pred_label, update_target_image_pred_probsval, update_target_image_pred_classname = updatepredition(prev_image, current_image, cur_target_image_pred_label, cur_target_image_pred_probsval, cur_target_image_pred_classname, prev_target_image_pred_label, prev_target_image_pred_probsval, prev_target_image_pred_classname)

              update_target_image_pred_label = cur_target_image_pred_label
              update_target_image_pred_probsval = cur_target_image_pred_probsval
              update_target_image_pred_classname = cur_target_image_pred_classname

              frame_with_dialogue = add_dialogue_box(current_frame, f'Frame: {current_frame_count}, Predicted: {update_target_image_pred_label}, Class:{update_target_image_pred_classname}')


              with open('/python_programming/FYP_2022/viadnet/test_video_output/dove/updatepredict_dove_video.csv', 'a', newline ='') as f:
                 writer = csv.writer(f)
                 data = [prev_frame_count, prev_target_image_pred_label, prev_target_image_pred_probsval, prev_target_image_pred_classname,  mean_brightness1, current_frame_count, cur_target_image_pred_label, update_target_image_pred_label, update_target_image_pred_probsval, update_target_image_pred_classname, mean_brightness2, mean_brightness_diff, variance_brightness1, variance_brightness2,
                     variance_brightness_diff, skewness_brightness1, skewness_brightness2, skewness_brightness_diff,  hist_diff]
                 writer.writerow(data)

    '''
    else :
          frame_with_dialogue = add_dialogue_box(current_frame, f'Frame: {current_frame_count}, Predicted: {update_target_image_pred_label}, Class:{update_target_image_pred_classname}')

          with open('/python_programming/FYP_2022/viadnet/test_video_output/dove/updatepredict_dove_video.csv', 'a', newline ='') as f:
              writer = csv.writer(f)
              data = [prev_frame_count, prev_target_image_pred_label, prev_target_image_pred_probsval, prev_target_image_pred_classname,  mean_brightness1, current_frame_count, cur_target_image_pred_label, update_target_image_pred_label, update_target_image_pred_probsval, update_target_image_pred_classname, mean_brightness2, mean_brightness_diff, variance_brightness1, variance_brightness2,
                variance_brightness_diff, skewness_brightness1, skewness_brightness2, skewness_brightness_diff,  hist_diff]
              writer.writerow(data)
    '''


    frame_count += 1

    # Update the previous frame for the next iteration
    prev_frame = current_frame
    prev_target_image_pred_label = cur_target_image_pred_label#update_target_image_pred_label
    prev_target_image_pred_probsval = cur_target_image_pred_probsval#update_target_image_pred_probsval
    prev_target_image_pred_classname = cur_target_image_pred_classname#update_target_image_pred_classname


    prev_frame_count = current_frame_count
    current_frame_count += 1


Streaming output truncated to the last 5000 lines.
current frame count: 2133
Saving frame2132  with mean brightness 101.909
Saving frame2133  with mean brightness 100.364
image name: 2133 | Pred label: 1 | pred prob: 0.788 | class name: No-billboard
frame_count: 2133 | previmagefile 2133 | prev_target_image_pred_label: 1 | prev_predval: 0.611 | prev_target_image_pred_classname: No-billboard | mean_brightness1: 100.364 | curimagefile: 2134 | cur_target_image_pred_label: 1 | cur_predval: 0.788 | cur_target_image_pred_classname: No-billboard | mean_brightness2: 98.817 | mean_brightness_diff: 1.547 hist_diff: 0.9949570345856384
frame_count: 2133
prev frame count: 2133
current frame count: 2134
Saving frame2133  with mean brightness 100.364
Saving frame2134  with mean brightness 98.817
image name: 2134 | Pred label: 1 | pred prob: 0.683 | class name: No-billboard
frame_count: 2134 | previmagefile 2134 | prev_target_image_pred_label: 1 | prev_predval: 0.788 | prev_target_image_pred_classname

C:\Users\sukri\AppData\Local\Temp\ipykernel_13424\309886016.py:16: RuntimeWarning: invalid value encountered in double_scalars
  skewness_brightness2 = round(np.mean((gray2 - mean_brightness2) ** 3) / (np.var(gray2) ** (3/2)),3)


image name: 2846 | Pred label: 1 | pred prob: 0.854 | class name: No-billboard


error: ignored

In [ ]:
data_pred = pd.read_csv('/python_programming/FYP_2022/viadnet/test_video_output/dove/updatepredict_dove_video.csv')
data_act = pd.read_csv('/python_programming/FYP_2022/viadnet/test_video_output/actualdovelabel.csv')

# Assuming your CSV file has columns named 'actual' and 'predicted'
actual = data_act['Actual label'].values
updated_predicted = data_pred['update_pred_label'].values
#preicted = data_pred

# Finding precision and recall
accuracy = accuracy_score(actual, predicted)
print("Accuracy   :", accuracy)
precision = precision_score(actual, predicted)
print("Precision :", precision)
recall = recall_score(actual, predicted)
print("Recall    :", recall)
F1_score = f1_score(actual, predicted)
print("F1-score  :", F1_score)